In [ ]:
# This has to go in its own cell or it screws up the defaults we'll set later
%matplotlib inline

In [ ]:
import numpy as np
import musictoys
import musictoys.audiofile
import musictoys.analysis
import musictoys.spectral
from scrapbook import plot

In [ ]:
filedata, filerate = musictoys.audiofile.read("audio_files/kronfeld-dreamatic.wav")
#filedata, filerate = musictoys.audiofile.read("audio_files/jfb-back_home.wav")
#filedata, filerate = musictoys.audiofile.read("audio_files/liberty_chaps-get_up_get_down.wav")
samples, samplerate = musictoys.analysis.normalize(filedata, filerate)

In [ ]:
frame_size = 1024
step_size = frame_size / 2
step_rate = samplerate / step_size
num_frames = int((len(samples) - frame_size + 1) / step_size)
graph_height = 256
gridx, gridy = np.mgrid[0:num_frames, 0:graph_height]

In [ ]:
mag2d = np.zeros((num_frames, graph_height))
for i in range(num_frames):
    clip = samples[i*step_size:i*step_size+frame_size]
    hist, bin_edges = np.histogram(clip, bins=graph_height, range=(-1,1))
    mag2d[i] = hist / np.float(hist.max())
# normalize each frame's samples: divide by its max value
# take sqrt to represent power with brightness
power2d = np.sqrt(mag2d)
plot.gram(power2d, x=step_rate)

In [ ]:
hann = np.hanning(frame_size)

spectrogram = []
for i in range(num_frames):
    # get the clip and weight according to the hann window
    clip = hann * samples[i*step_size:i*step_size+frame_size]
    # take the FFT and discard the symmetrical components
    spectrum = np.fft.rfft(clip)#[:frame_size/2]
    # add this spectrum to the series of spectra
    spectrogram.append(spectrum)

spectrogram = np.array(spectrogram)
# take absolute value to get magnitude
spectrogram = np.absolute(spectrogram) / np.sum(hann)
# square to get power
powerspectrogram = np.square(spectrogram)
# convert to decibels, clip noise floor
loudnessgram = (10 * np.log10(powerspectrogram)).clip(-120)
# compute the center frequency of each FFT bin
nyquist_freq = samplerate / 2.0
fft_bin_freq = np.arange(spectrogram.shape[1]) * nyquist_freq / float(spectrogram.shape[1])
half_nyquist = nyquist_freq / 2.0

In [ ]:
plot.gram(loudnessgram, x=step_rate, y=fft_bin_freq, cmap='plasma')

In [ ]:
centroids = musictoys.spectral.centroid(powerspectrogram, samplerate)
plot.line(centroids, x=step_rate)

In [ ]:
spreads = musictoys.spectral.spread(spectrogram, samplerate)
plot.line(spreads, x=step_rate)

In [ ]:
crests = musictoys.spectral.crest(spectrogram)
plot.line(crests, x=step_rate)


In [ ]:
spectrum2d = np.zeros((num_frames, graph_height))
# generate a normal distribution representing centroid & spread
for i in range(num_frames):
    centroid = centroids[i]
    spread = spreads[i]
    distribution = centroid + spread * np.random.randn(10000)
    hist, bin_edges = np.histogram(distribution, bins=graph_height, range=(0,half_nyquist))
    spectrum2d[i] = hist / np.float(hist.max())
plot.gram(spectrum2d, x=step_rate, cmap='plasma')

In [ ]:
hues = centroids / half_nyquist
hues /= hues.max()
plot.line(np.sqrt(hues), x=step_rate)

In [ ]:
saturations = 1.0 - (crests / crests.max())
plot.line(saturations, x=step_rate)
# yes! this is the one

In [ ]:
# generate corresponding hue and saturation matrices
hue2d = hues[gridx]
plot.gram(hue2d, x=step_rate, cmap='rainbow_r')

In [ ]:
sat2d = saturations[gridx]
plot.gram(sat2d, x=step_rate)


In [ ]:
def hsl_to_rgb(h, s, l):
    shape = h.shape
    
    # all inputs and outputs range 0..1
    r = np.zeros(shape)
    g = np.zeros(shape)
    b = np.zeros(shape)
    
    # where the color is totally desaturated, use only luminance
    grey = (s==0)
    r[grey] = l[grey]
    g[grey] = l[grey]
    b[grey] = l[grey]
    
    # scale the saturation differently around medium luminance
    q = np.zeros(shape)
    low_luma = l < 0.5
    q[low_luma] = l[low_luma] * (1 + s[low_luma])
    hi_luma = l >= 0.5
    q[hi_luma] = l[hi_luma] + s[hi_luma] - l[hi_luma] * s[hi_luma]
    # the other hue factor is proportional
    p = 2 * l - q;
    
    def channel(t):
        # enforce limits
        t[t < 0] += 1.0
        t[t > 1] -= 1.0
        x = np.zeros(shape)
        tA = t < 1/6.
        x[tA] = p[tA] + (q[tA] - p[tA]) * 6 * t[tA] 
        tB = (t >= 1/6.) & (t < 1/2.)
        x[tB] = q[tB]
        tC = (t >= 1/2.) & (t < 2/3.)
        x[tC] = p[tC] + (q[tC] - p[tC]) * (2/3. - t[tC]) * 6
        tD = t >= 2/3.
        x[tD] = p[tD]
        return x

    chroma = (s != 0)
    r[chroma] = channel(h + 1./3.)[chroma]
    g[chroma] = channel(h)[chroma]
    b[chroma] = channel(h - 1./3.)[chroma]

    return r, g, b

r2d, g2d, b2d = hsl_to_rgb(hue2d, sat2d, mag2d)
image_rgb = np.stack((r2d, g2d, b2d), axis=2)
plot.colorgram(image_rgb, x=step_rate)
